In [ ]:
pip install tensorflow


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate

def conv_block(input_tensor, num_filters):
    tensor = Conv2D(num_filters, (3, 3), padding='same')(input_tensor)
    tensor = tf.keras.layers.Activation('relu')(tensor)
    tensor = Conv2D(num_filters, (3, 3), padding='same')(tensor)
    tensor = tf.keras.layers.Activation('relu')(tensor)
    return tensor

def encoder_block(input_tensor, num_filters):
    tensor = conv_block(input_tensor, num_filters)
    pool = MaxPooling2D((2, 2))(tensor)
    return tensor, pool

def decoder_block(input_tensor, concat_tensor, num_filters):
    tensor = UpSampling2D((2, 2))(input_tensor)
    tensor = concatenate([tensor, concat_tensor], axis=-1)
    tensor = conv_block(tensor, num_filters)
    return tensor

def unet_model(input_shape):
    inputs = Input(input_shape)

    encoder0_pool, encoder0 = encoder_block(inputs, 32)
    encoder1_pool, encoder1 = encoder_block(encoder0_pool, 64)
    encoder2_pool, encoder2 = encoder_block(encoder1_pool, 128)
    encoder3_pool, encoder3 = encoder_block(encoder2_pool, 256)

    center = conv_block(encoder3_pool, 512)

    decoder3 = decoder_block(center, encoder3, 256)
    decoder2 = decoder_block(decoder3, encoder2, 128)
    decoder1 = decoder_block(decoder2, encoder1, 64)
    decoder0 = decoder_block(decoder1, encoder0, 32)

    outputs = Conv2D(1, (1, 1), activation='sigmoid')(decoder0)

    model = Model(inputs=[inputs], outputs=[outputs])
    return model

# Set the input shape to match your dataset. E.g., (256, 256, 1) for 256x256 grayscale images.
input_shape = (256, 256, 1)
model = unet_model(input_shape)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()
